In [610]:
import numpy as np
import pandas as pd
import math
import json
import random
# setup data
POPULATION_SIZE = 200
PERCENT_SELECTION = .2
PERCENT_OLD_PENALTY = .1

def seededRandom(a, b): 
    res = random.randrange(a, b)
    seed = random.randrange(0, 666666)
    random.seed(seed, 2)

class Sack:
    def __init__(self, wLimit, sLimit):
        self.sack = []
        self.w = 0
        self.s = 0
        self.wLimit = wLimit
        self.sLimit = sLimit

    def push(self, item): 
        weight, size, value = item
        if (weight + self.w > self.wLimit) or (size + self.s > self.sLimit): 
            return False
        self.sack.append(item)
        self.w += weight
        self.s += size
        return True

    def getItems(self):
        return self.sack
    def getLength(self):
        return len(self.sack)
    def getSum(self, i):
        value = 0
        for item in self.sack:
            value += item[i]
        return value

        
def generatePopulationForSack(size, genes, wLimit, sLimit, data):
    population = []
    for i in range(0, size):
        sack = Sack(wLimit, sLimit)

        start = seededRandom(0, genes)
        shiftedData =data[start:] + data[0:start]
        fits = 0
        for item in shiftedData: 
            result = sack.push(item)
            if result: 
                fits+=1
            else:
                break
        temp = [1] * fits + [0] * (genes - fits)
        population.append(temp[genes - start:] + temp[0:genes - start])

    return population
def calcValues(population, items, wLimit, sLimit):
    values = []
    for ind in population:
        sack = Sack(wLimit, sLimit)
        broken = False
        for (gene, item) in zip(ind, items):
            if gene > 0: 
                if not sack.push(item): broken = True
        values.append(0 if broken else sack.getSum(2))
    return values

def toEvaled(population, items, wLimit, sLimit):
    return list(zip(calcValues(population, items, wLimit, sLimit), population))

def selectBest(evaledInd, p):
    sortedInd = sorted(evaledInd, key=lambda x: x[0], reverse=True)
    return sortedInd[0: int(len(evaledInd)*p)]

def dropValues(evaled):
    return list(map(lambda x: x[1], evaled))

def breed(population):
    mothers = population[0::2] * 2
    fathers = population[1::2] * 2
    children = []
    for (m, f) in zip(mothers, fathers):
        child = list(map(lambda x: x[seededRandom(0, 2)], zip(m, f)))
        children.append(child)
    return children

def mutate(children):
    index = seededRandom(0, len(children))
    children[index] = list(map(lambda x: (x + 1) % 2, children[index]))

def selectNewPopulation(oldEvaled, newEvaled, oldPenalty):
    oldWithPenalty = list(map(lambda x: (x[0] * (1 - oldPenalty), x[1]), oldEvaled))
    bothGenerations = oldWithPenalty + newEvaled
    return selectBest(bothGenerations, len(oldWithPenalty) / len(bothGenerations))

def generationCycle(population, items, wLimit, sLimit):
    evaledParents = toEvaled(population, items, wLimit, sLimit)
    best = dropValues(selectBest(evaledParents, PERCENT_SELECTION))
    random.shuffle(best)
    # print(best)
    children = breed(best)
    # print(children)
    mutate(children)
    evaledChildren = toEvaled(children, items, wLimit, sLimit)
    # print(evaledChildren)
    newPopulation = selectNewPopulation(evaledParents, evaledChildren, PERCENT_OLD_PENALTY)
    return dropValues(newPopulation)
def evalResult(oldPopulation, newPopulation):
    return 1

In [611]:

raw = pd.read_csv('19.txt', sep = ' ', names = ['w','s','v'], header=None)
weightLimit, sizeLimit, *_ = list(raw.iloc[0])
raw = raw.drop([0])
itemsCount = raw.shape[0]
smallestValue = raw['v'].min()
print(weightLimit, sizeLimit, smallestValue)
items = []
for i in range(0, itemsCount):
  items.append(tuple(raw.iloc[i]))
# items - array of tuple
# 0 - weight
# 1 - size
# 2 - value
population = generatePopulationForSack(POPULATION_SIZE, itemsCount, weightLimit, sizeLimit, items)

prevBest = 0
genCount = 0
while(1):
    population = generationCycle(population, items, weightLimit, sizeLimit)
    evaled = toEvaled(population, items, weightLimit, sizeLimit)
    best = selectBest(evaled, 1 / POPULATION_SIZE)[0]
    print(best[0] - prevBest)
    genCount+=1
    if best[0] - prevBest <= smallestValue or genCount >= 50: break
    prevBest = best[0]
    
print(best)
   

13000.0 12.0 108.0
4368.0
1.0
(4369.0, [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0])
